<a href="https://colab.research.google.com/github/carlosvalenciano/Project4/blob/main/Stock_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [634 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,572 kB]
Get:13 http://archive.ub

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time
import datetime as dt

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the csv into a DataFrame.

df = spark.read.format("csv").option("header", "true").load("Multiple_Stocks.csv")
df.show()


+----------+-----------+-----------+-----------+-----------+-----------+---------+------------+------------+---------------+
|      Date|       Open|       High|        Low|      Close|  Adj_Close|   Volume|company_name|Scaled_Close|predicted_close|
+----------+-----------+-----------+-----------+-----------+-----------+---------+------------+------------+---------------+
|2019-12-12|66.94499969|68.13999939|66.83000183|67.86499786|66.11624908|137310400|       APPLE| 0.045575656|    72.47750092|
|2019-12-13|67.86499786|68.82499695|67.73249817|68.78749847|67.01499176|133587600|       APPLE| 0.048372223|    72.44999695|
|2019-12-16|      69.25|70.19750214|69.24500275|69.96499634|68.16214752|128186000|       APPLE| 0.051941815|    72.87999725|
|2019-12-17|69.89250183|70.44249725|69.69999695|70.10250092|68.29610443|114158400|       APPLE| 0.052358661|    73.41249847|
|2019-12-18|69.94999695|70.47499847|69.77999878|69.93499756|68.13290405|116028400|       APPLE| 0.051850873|    75.08750153|


In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('stocks')


In [9]:
# Calculated the Predicted Close for the next three days
spark.sql("""
SELECT
  company_name AS Company,
  (Avg(predicted_close)) AS Predicted_Close
  FROM stocks
  WHERE Date BETWEEN ('2023-12-07') AND ('2023-12-11')
  GROUP BY Company
  ORDER BY Predicted_Close DESC
""").show()

+---------+------------------+
|  Company|   Predicted_Close|
+---------+------------------+
|MICROSOFT|       373.3289693|
|     META| 328.2066650333333|
|    APPLE|194.51652016666665|
|   AMAZON|       145.0767466|
|   GOOGLE|133.76257833333332|
+---------+------------------+



In [55]:
# Dataframe that tell whether the stocks went down or up based on the open and close for the day
spark.sql("""
SELECT
Date,
company_name,
Open,
Close,
CASE WHEN (close < open) THEN 'DOWN'
     WHEN (close > open) THEN 'UP'
 	   ELSE '0'
END AS Direction
FROM stocks
WHERE company_name == 'META'
""").show()

+----------+------------+-----------+-----------+---------+
|      Date|company_name|       Open|      Close|Direction|
+----------+------------+-----------+-----------+---------+
|2019-12-12|        META|202.3500061|     196.75|     DOWN|
|2019-12-13|        META|196.3999939|194.1100006|     DOWN|
|2019-12-16|        META|195.2700043|197.9199982|       UP|
|2019-12-17|        META|198.8399963|198.3899994|     DOWN|
|2019-12-18|        META|200.0899963|      202.5|       UP|
|2019-12-19|        META|202.7799988|206.0599976|       UP|
|2019-12-20|        META|207.4799957|206.3000031|     DOWN|
|2019-12-23|        META|206.6999969|206.1799927|     DOWN|
|2019-12-24|        META|206.3000031|205.1199951|     DOWN|
|2019-12-26|        META|205.5700073|207.7899933|       UP|
|2019-12-27|        META|208.6699982|208.1000061|     DOWN|
|2019-12-30|        META|207.8600006|204.4100037|     DOWN|
|2019-12-31|        META|        204|     205.25|       UP|
|2020-01-02|        META|     206.75|209

In [35]:
# Calculated the Yearly Change for 2023
spark.sql("""
SELECT
company_name AS Company,
(Avg(predicted_close)) * 100) AS Yearly_Change
FROM stocks
WHERE Year(Date) == 2023
GROUP BY company_name
ORDER BY Yearly_Change DESC
""").show()

+---------+------------------+
|  Company|     Yearly_Change|
+---------+------------------+
|     META|164.84285913900715|
|   AMAZON| 70.71144508605813|
|MICROSOFT| 52.74806033015259|
|   GOOGLE|49.949898871108594|
|    APPLE|48.280622105747234|
+---------+------------------+



In [36]:
# Calculated the Yearly Change for 2022
spark.sql("""
SELECT
company_name AS Company,
(((LAST_VALUE(Close) - FIRST_VALUE(Open))/FIRST_VALUE(Open)) * 100) AS Yearly_Change
FROM stocks
WHERE Year(Date) == 2022
GROUP BY company_name
ORDER BY Yearly_Change DESC
""").show()


+---------+-------------------+
|  Company|      Yearly_Change|
+---------+-------------------+
|    APPLE|-26.935842442307155|
|MICROSOFT| -28.48665485681051|
|   GOOGLE| -38.58473803568448|
|   AMAZON|   -49.865712655424|
|     META| -64.42802227615097|
+---------+-------------------+

